In [1]:
#all modules importation
import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd
import csv
import re
import time
from selenium.common.exceptions import NoSuchElementException
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# specify the url
urlpage = 'https://transparency.entsoe.eu/generation/r2/actualGenerationPerGenerationUnit/show'
print(urlpage)

https://transparency.entsoe.eu/generation/r2/actualGenerationPerGenerationUnit/show


In [3]:
# run firefox webdriver from executable path 
driver = webdriver.Firefox(executable_path = 'geckodriver.exe')

#wait to mozila Firefox to open 
#please do not click on any button in that page !

In [4]:
# get web page
driver.get(urlpage)

In [5]:
# check "I Agree" button
def Check_agree(driver):
    try:
        submit_button = driver.find_elements_by_xpath('//*[@id="close-button"]')[0]
        err =0
        submit_button.click()
    except:
        err =1

In [6]:
Check_agree(driver)

In [7]:
#for some region such as some region in Germany no data is available so we should handle this problem
#for now we will deal with countries that display one region such as France (FR)
def NoData():
    if ("No data found for criteria selected" == driver.find_elements_by_xpath("/html/body/div[2]/section/form/div[3]/div[2]/div[3]/div/div/table/tbody/tr/td")[0].text):
        print("choose another region please, we don't support countries with multiple regions where some dosen't display data")

In [8]:
def selectChild(k):
    regions = []
    i = 1
    while(1):
        try:
            region = driver.find_elements_by_xpath("/html/body/div[2]/section/form/div[3]/div[1]/div[1]/div[2]/div["+str(k)+"]/div[2]/div["+ str(i) +"]/label")[0].text
            i += 1
            regions.append(region)
        except:
            break
    return regions

In [10]:
def checkCapital():
    k = 0
    while(1):
        k += 1
        try:
            regionbox = driver.find_element_by_xpath('//*[@id="'+ str(k) +'"]').is_selected()
            regionname = driver.find_element_by_xpath("/html/body/div[2]/section/form/div[3]/div[1]/div[1]/div[2]/div["+ str(k) +"]/div[1]/label")
            if regionbox:
                 return(regionname.text)
        except NoSuchElementException:
            return None

In [12]:
def PickCapital2(myCapital):
    actual = checkCapital()
    if actual == myCapital:
        print("the country you want to select, ",myCapital,", is already picked!")
        return
    k = 0
    while(1):
        k += 1
        try : 
            capital = driver.find_elements_by_xpath("/html/body/div[2]/section/form/div[3]/div[1]/div[1]/div[2]/div[" + str(k) + "]/div[1]/label")[0]
            try :
                if(myCapital == capital.text):
                    capital.click()
                    regions = selectChild(k)
                    if len(regions) == 1:
                        print("this country have one region, " + capital.text + ", so we are choosing it by default")
                    else:
                        for index in range(len(regions)):
                            print("( "+ str(index + 1) +" )" + regions[index])
                        choice = int(input("This country have more then a region,as displayed, please choose one by entering its number:"))
                        driver.find_elements_by_xpath("/html/body/div[2]/section/form/div[3]/div[1]/div[1]/div[2]/div["+str(k)+"]/div[2]/div["+ str(choice) +"]/div/a")[0].click()
                    print("Done")
                    #NoData()
                    break
            except:
                print("somthing went wrong, please try again")
                break
        except:
            print("Please enter a valid  Area name !")
            break
    return

In [39]:
PickCapital2('Germany (DE)')

the country you want to select,  Germany (DE) , is already picked!


In [162]:
#picking the desired country 
def PickCapital(myCapital):
    k = 0
    while(1):
        k += 1
        try : 
            capital = driver.find_elements_by_xpath("/html/body/div[2]/section/form/div[3]/div[1]/div[1]/div[2]/div[" + str(k) + "]/div[1]/label")[0]
            try :
                if(myCapital == capital.text):
                    capital.click()
                    print("Done")
                    NoData()
                    break
            except:
                print("somthing went wrong, please try again")
                break
        except:
            print("Please enter a valid  Area name !")
            break
    return

In [17]:
#picking the desired Generation U
def PickGeneration(myGene):
    k = 0
    while(1):
        k += 1
        try:
            gene = driver.find_elements_by_xpath("/html/body/div[2]/section/form/div[3]/div[2]/div[3]/div/div/table/tbody/tr["+ str(k) +"]/td[2]")[0]
            try :
                if(myGene == gene.text):
                    plus = driver.find_elements_by_xpath("/html/body/div[2]/section/form/div[3]/div[2]/div[3]/div/div/table/tbody/tr["+ str(k) +"]/td[5]/a")[0]
                    plus.click()
                    print("Done clicking plus button")
                    path1 = "/html/body/div[2]/section/form/div[3]/div[2]/div[3]/div/div/table/tbody/tr["+ str(k) +"]/td[5]/a"
                    #clickin 100 button to display all day data
                    p = 0
                    while(1):
                        time.sleep(.5)
                        #wait for the table to be rendered
                        try:
                            p += 1
                            expand = driver.find_elements_by_xpath("/html/body/div[2]/section/form/div[3]/div[2]/div[3]/div/div/table/tbody/tr["+ str(k+1) +"]/td/div/div/div[2]/div[1]/a[4]")[0]
                            expand.click()
                            print("expand successfully")
                            path2 = "/html/body/div[2]/section/form/div[3]/div[2]/div[3]/div/div/table/tbody/tr["+ str(k+1) +"]/td/div/div/div[2]/div[1]/a[4]"
                            path3= "/html/body/div[2]/section/form/div[3]/div[2]/div[3]/div/div[1]/table/tbody/tr["+ str(k+1) +"]"
                            break
                        except:
                            print("bad connection, waited " + str(p * 0.5) + "seconds")
                            if (p >= 10):
                                print("couldn't expand !")
                                break
                    return ([path1,path2,path3])
            except:
                print("somthing went wrong, please try again")
                break
        except:
            print("Please enter a valid  Generation Unit name !")
            break

In [18]:
# Please enter a valid Generation Unit Name so we can continue for example :BOUCHAIN 7,ARRIGHI 2,CORDEMAIS 5....
# each country(capital) has its own Generation Unit,thus the examples given here aren't alwayse present(they are for France)
path = PickGeneration("Emsland A")

Done clicking plus button
expand successfully


In [19]:
#check if an error has occured, in particular the server has shut down

def isError(totalDays,days,driver):
    try :
        error_button = driver.find_elements_by_xpath('/html/body/div[6]/div[3]/div/button')[0]
        error_button.click()
        print("An error has occured on the ",totalDays - days ,"day")
        time.sleep(7)
    except:
        x =1
# verify that the page is fully loaded
def isloaded(driver) :
    try:
        python_button1 = driver.find_elements_by_xpath("/html/body/div[2]/section/form/div[3]/div[2]/div[3]/div/div[1]/table/tbody/tr[1]")[0]
        notloaded = False
    except:
        notloaded = True
    return(notloaded)
# getting all the data needed
def Get_Data(driver, days,path):
    totalDays = days
    output_rows = []
    if NoData():
        print("No data found for criteria selected, take a look at the website, it contains NO DATA")
        return
    while(days):
        try:
            isError(totalDays,days,driver)
            Check_agree(driver)
            days-=1
            # return to yesterday's data
            submit_button = driver.find_elements_by_xpath('//*[@id="date-arrow-left-user-friendly-input-dv-date-from"]')[0]
            submit_button.click()
            time.sleep(2)
            # extract data from its node
            while(isloaded(driver)):
                time.sleep(0.5)
            isError(totalDays,days,driver)
            python_button1 = driver.find_elements_by_xpath(path[0])[0]
            python_button1.click()
            isError(totalDays,days,driver)
            # print out all data of the day
            python_button1 = driver.find_elements_by_xpath(path[1])[0]
            python_button1.click()
            isError(totalDays,days,driver)
            time.sleep(0.5)
            # get data 's date
            html_date = driver.find_elements_by_xpath('//*[@id="originalinput-wrapper-dv-date-from"]')[0].get_attribute('innerHTML')        
            Date = BeautifulSoup(html_date, 'html.parser')
            match = re.search(r'\d{2}.\d{2}.\d{4}', str(Date))
            Date = match.group()
            # Get table in html
            html_table1 =driver.find_elements_by_xpath(path[2])[0]
            soup1 = BeautifulSoup(html_table1.get_attribute('innerHTML'), 'html.parser')
            # make table
            if(output_rows != None):
                output_rows =output_rows +  makeTable(soup1,Date)
            print("we checked for now ", totalDays - days , "out of ",totalDays)
            #the server is not as good as it should be so if we keep requesting pages it may shut down
            time.sleep(2)
        except:
            try:
                isError(totalDays,days,driver)
            except:
                print("we still haven't check ",days + 1," days!")
                return([output_rows], days + 1)
    time.sleep(1)
    return([output_rows],days)
# make out a table from html ;
def makeTable(soup,Date):
    output_rows =[]
    table = soup.find("table")
    for table_row in soup.findAll('tr'):
        columns = table_row.findAll('td')
        output_row = []
        for column in columns:
              output_row.append(column.text)
        output_row.append(Date)
        if(output_row[0] != Date):
            output_rows.append(output_row)
    return(output_rows)

In [20]:
# please indicate the number of days you want to scrap ! for exemple 90 here
# it will take some times this is due to the fact that the website's server is not quite good to support a lot of requests ;
#so I, on purpose, made the script go slowly !
(table,nbr) = Get_Data(driver, 50,path)

we checked for now  1 out of  50
we checked for now  2 out of  50
we checked for now  3 out of  50
we checked for now  4 out of  50
we checked for now  5 out of  50
we checked for now  6 out of  50
we checked for now  7 out of  50
we checked for now  8 out of  50
we checked for now  9 out of  50
we checked for now  10 out of  50
we checked for now  11 out of  50
we checked for now  12 out of  50
we checked for now  13 out of  50
we checked for now  14 out of  50
we checked for now  15 out of  50
we checked for now  16 out of  50
we checked for now  17 out of  50
we checked for now  18 out of  50
we checked for now  19 out of  50
we checked for now  20 out of  50
we checked for now  21 out of  50
we checked for now  22 out of  50
we checked for now  23 out of  50
we checked for now  24 out of  50
we checked for now  25 out of  50
we checked for now  26 out of  50
we checked for now  27 out of  50
we checked for now  28 out of  50
we checked for now  29 out of  50
we checked for now  30 

In [26]:
len(table[0])

1080

In [22]:
nbr

0

In [27]:
table = pd.DataFrame(table[0], columns =['MTU', 'Generation','Consumption','Date'])

In [28]:
table.to_csv(r'./data_table.csv')

In [29]:
# read one of the saved files
data_table = pd.read_csv("data_table.csv") 

In [35]:
# print the header of this file
data_table.head()

,Unnamed: 0,MTU,Generation,Consumption,Date
0,0,00:00 - 01:00,-,-,02.10.2019
1,1,01:00 - 02:00,-,-,02.10.2019
2,2,02:00 - 03:00,-,-,02.10.2019
3,3,03:00 - 04:00,-,-,02.10.2019
4,4,04:00 - 05:00,-,-,02.10.2019


In [36]:
#Get the unique numbers in Generation
data_table.Generation.nunique()

78